### 1- Create stats masks - Electrodes to be plotted
    Elec that reach the threshold in time (min sig)

In [166]:
"""
Correct statistics by region for all freqs, electrodes and subjects
"""
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product
import seaborn as sns
import pandas as pd

conds = ['low','high']
exp = 'E'
met = 'zFisher' #k=10 zFisher
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_clf_pow_sklearn_time_'+exp+'_by_cond/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = '{}_sources_{}_'+conds[0]+'_'+conds[1]+'_bsl_3s.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_minwin{}_th{}_'+met+'_by_roi.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ']
#freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
freqs = ['0_theta','1_alpha','2_beta','3_gamma']
rois_to_keep = ['ACC','aHC','IFG','Ins','MFG','OFC_olf','PHG','SFG','pPirT','HC','Amg']
#rois_to_keep = ['aHC','IFG','OFC_olf','pPirT','pHC']

thrs = [0.05,0.01,0.001]
n_perms = 1000

# #Find threshold by subject
for t in thrs:
    th_roi = []
    for roi in rois_to_keep:
        th_freq_roi = []
        for freq in freqs:
            mat = np.load(f_form.format('All_subjects',freq))
            id_lab = [i for i,x in enumerate(mat['s_labels']) if x == roi]
            s_perm = mat['s_perm'][id_lab].swapaxes(0,-1)
            if s_perm.shape[1] > 0:
                th_perm = perm_pvalue2level(s_perm, p=t, maxst=True)[0]
            else:
                s_perm = s_perm[:,np.newaxis]
                th_perm = perm_pvalue2level(s_perm, p=t, maxst=True)[0]
            th_freq_roi.append(th_perm)
        th_roi.append(np.max(th_freq_roi))
    print('perm max size',t, th_roi)

    df = pd.read_csv(path_npz+'1_all_subjects_info_elecs_AUC_Pow_Time.csv')
    df_sel = df.loc[df['s_Mai_RL'].isin(rois_to_keep)]
    dic_th_roi = {}
    for i,roi in enumerate(rois_to_keep):
        dic_th_roi[roi] = th_roi[i]
    df_sel['thr'] = df_sel['s_Mai_RL'].map(dic_th_roi)
    for freq in freqs:
        #1 if sig 0 if not
        df_sel['sig_'+freq] = [0 if auc <= th else 1 for auc, th in zip(df_sel[freq+'_AUC'],df_sel['thr'])]
    df_sel.to_csv(path_npz+'2_all_subjects_info_elecs_AUC_by_roi_'+str(t)+'_corr.csv')

-> Olfacto loaded
perm max size 0.05 [0.8666666666666668, 0.9333333333333332, 0.8666666666666668, 0.8777777777777779, 0.8666666666666668, 0.8555555555555555, 0.8777777777777779, 0.9, 0.8277777777777778, 0.8222222222222222, 0.8777777777777779]
perm max size 0.01 [0.95, 1.0, 0.9333333333333333, 0.9555555555555555, 0.9666666666666668, 0.9333333333333332, 0.95, 0.9666666666666668, 0.9111111111111111, 0.9, 0.9555555555555555]
perm max size 0.001 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [23]:
"""No rois to select because rois already selcted when creating npz"""
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product
import seaborn as sns

conds = ['low','high']#['low','high']
exps = ['E']#,'R','E_R']
meth = 'pow'
#met = 'zFisher' #k=10 zFisher
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'classified/0_clf_pow_sklearn_time_{}_by_cond/figure/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = join(path_npz, '{}_sources_{}_'+conds[0]+'_'+conds[1]+'_physFT.npz')
f_save = '{}_mask_stat_{}_th={}_win={}_not_corr_freqs.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
subjects = ['FERJ','LEFC','SEMC','PIRJ','VACJ','CHAF'] #CHAF
freqs = ['gamma_high']

thrs = [0.05,0.01]
n_perms = 1000
min_win = 4

# #Find threshold by subject
for th, exp in product(thrs,exps):
    th_final = []
    for su in sorted(subjects):
        for freq in freqs:
            mat = np.load(f_form.format(exp,'All_subjects',freq))
            id_su = np.where(mat['su_codes']==su)[0]
            s_perm = mat['s_perm'][id_su].swapaxes(0,-1)
            if s_perm.shape[1] > 0:
                th_perm = perm_pvalue2level(s_perm, p=th, maxst=True)[0]
                th_perm = np.max(th_perm)
            else:
                s_perm = s_perm[:,np.newaxis]
                th_perm = perm_pvalue2level(s_perm, p=th, maxst=True)[0]
            th_final.append(th_perm)
    print('perm max size',th,'>>>',th_final)
    print('subjects',[su for su in sorted(subjects)])

    # Create masks for a specific perm threshold
    for f,freq in enumerate(freqs):
        mat = np.load(f_form.format(exp,'All_subjects',freq))
        mask_f = np.array([])
        for i,su in enumerate(sorted(subjects)):
            id_su = np.where(mat['su_codes']==su)
            if np.size(mat['s_da'][1]) > 1:
                s_da = mat['s_da'][id_su]
            else:
                s_da = mat['s_da'][id_su][:,np.newaxis]
            n_elecs, npts = s_da.shape
            mask_su = []
            for elec in range(n_elecs):
                da_elec = s_da[elec]
                pvals = np.array([])
                for t in range(npts):
                    if da_elec[t] > th_final[i]:
                        pval = 1
                    if da_elec[t] <= th_final[i]:
                        pval = 0
                    pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
                underp = np.where(pvals == 1)[0]
                pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
                signif = [True for k in pvsplit if len(k) >= min_win]
                if len(signif) >=1:
                    remove = False
                else:
                    remove = True
                mask_su.append(remove)
            mask_f = np.hstack((mask_f, mask_su)) if np.size(mask_f) else mask_su
        np.save(f_save.format(exp,'All_subjects',freq,th,str(min_win)),mask_f)
    

-> Olfacto loaded
perm max size 0.05 >>> [0.9333333333333333, 0.775, 0.6978571428571428, 0.74, 0.7880952380952382, 0.8777777777777779]
subjects ['CHAF', 'FERJ', 'LEFC', 'PIRJ', 'SEMC', 'VACJ']
perm max size 0.01 >>> [1.0, 0.8583333333333334, 0.7661904761904761, 0.8111111111111111, 0.8666666666666668, 0.95]
subjects ['CHAF', 'FERJ', 'LEFC', 'PIRJ', 'SEMC', 'VACJ']


In [16]:
"""
SAVE masks for significant results
"""

from collections import Counter
import pandas as pd

min_win = 1
# Create masks for a specific perm threshold
for f,freq in enumerate(freqs):
    mat = np.load(f_form.format('All_subjects',freq))
    #df_labels = pd.read_csv(df_names)
    mask_f, labels = np.array([]),[]
    subj, power0, power1 = [], np.array([]), np.array([])
    num_elecs = []
    for i,su in enumerate(sorted(subjects)):
        id_su = np.where(mat['su_codes']==su)
        if np.size(mat['s_da'][1]) > 1 and freq in freqs:
            s_da = mat['s_da'][id_su]
        else:
            s_da = mat['s_da'][id_su][:,np.newaxis]
        n_elecs, npts = s_da.shape
        mask_su = []
        for elec in range(n_elecs):
            da_elec = s_da[elec]
            pvals = np.array([])
            for t in range(npts):
                if da_elec[t] > th_final[i]:
                    pval = 1
                if da_elec[t] <= th_final[i]:
                    pval = 0
                pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
            underp = np.where(pvals == 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= min_win]
            if len(signif) >=1:
                remove = False
            else:
                remove = True
            mask_su.append(remove)
        mask_f = np.hstack((mask_f, mask_su)) if np.size(mask_f) else mask_su
    np.save(f_save.format('All_subjects',freq,min_win,th),mask_f)
    

In [39]:
from brainpipe.statistics import perm_pvalue2level

def compute_th_by_su():
    th_by_su = []
    for su in sorted(subjects):
        th_su = []
        for freq in freqs:
            mat = np.load(f_form.format('All_subjects',freq,
                                        conds[0],conds[1]),allow_pickle=True)
            id_su = np.where(mat['su_codes']==su)[0]
            s_perm = mat['s_perm'][id_su].swapaxes(0,-1)
            th_perm = perm_pvalue2level(s_perm, p=th, maxst=True)[0]
            th_su.append(th_perm)
        th_by_su.append(np.max(th_su))
    return th_by_su

In [53]:
"""
Save mask for significant results + sum up results
"""
from collections import Counter
import numpy as np
import pandas as pd
from os.path import join
from brainpipe.system import study

st = study('Olfacto')
path_npz = join(st.path, 'classified/0_clf_pow_sklearn_time_'+exp+'_by_cond/figure/')
path_to_save = join(path_npz, 'HG_results_th={}_minwin={}/{}/')
f_form = '{}_sources_{}_{}_{}_physFT.npz'
f_form = join(path_npz, f_form)
csv_form = 'results_df_roi={}_sign={}_th={}_minwin={}.csv'
save_csv = join(path_to_save, csv_form)

th, exp = 0.05, 'E'
method, subjects = 'power', ['CHAF','FERJ','LEFC','PIRJ','SEMC','VACJ']
freqs, conds = ['gamma_high'], ['low','high']
min_win = 4

# Create masks for a specific perm threshold
for f,freq in enumerate(freqs):
    mat = np.load(f_form.format('All_subjects',freq,conds[0],conds[1]),allow_pickle=True)
    th_by_su = compute_th_by_su()
    mask_f, labels, channels = np.array([]),[], []
    subj, power0, power1 = [], np.array([]), np.array([])
    num_elecs, time_vec = [], []
    da_mean, da_max = [], []
    for i,su in enumerate(sorted(subjects)):
        id_su = np.where(mat['su_codes']==su)
        s_labels = mat['s_labels'][id_su]
        s_channels = mat['s_channels'][id_su]
        #print('before select ROIs',mat['s_da'].shape, np.size(mat['s_da'][1]))
        if np.size(mat['s_da'][1]) > 1 :
            s_da = mat['s_da'][id_su]
        else:
            s_da = mat['s_da'][id_su][:,np.newaxis]
        if method == 'power':
            pow0, pow1 = mat['s_elec_pow0'][id_su], mat['s_elec_pow1'][id_su]
        elif method == 'rsa':
            pow0, pow1 = mat['rsa0'][id_su], mat['rsa1'][id_su]
        n_elecs, npts = s_da.shape
        
        for elec in range(n_elecs):
            da_elec = s_da[elec]
            
            pvals = np.array([1 if da_elec[t] > th_by_su[i] else 0 for t in range(npts)])
            time = np.arange(-1025,2025,50)
            
            underp = np.where(pvals == 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [k for k in pvsplit if len(k) >= min_win]

            if len(signif) >= 1:
                idx_ = np.concatenate(signif)
                labels.append(s_labels[elec])
                channels.append(s_channels[elec])
                subj.append(su)
                pow0_, pow1_= np.mean(pow0[elec,idx_]), np.mean(pow1[elec,idx_])
                
                time_vec.append(time[idx_][0])
                da_mean.append(np.mean(da_elec[idx_]))
                da_max.append(np.max(da_elec[idx_]))
                num_elecs.append(elec)
                power0 = np.vstack((power0,np.round(pow0_,2))) if np.size(power0) \
                                                        else np.round(pow0_,2)
                power1 = np.vstack((power1,np.round(pow1_,2))) if np.size(power1) \
                                                        else np.round(pow1_,2)
    
    #SUM UP results
    subj = np.array(subj)[np.newaxis]
    labels = np.array(labels)[np.newaxis]
    channels = np.array(channels)[np.newaxis]
    num_elecs = np.array(num_elecs)[np.newaxis]
    time_vec = np.array(time_vec)[np.newaxis]
    da_mean = np.array(da_mean)[np.newaxis]
    da_max = np.array(da_max)[np.newaxis]
    x_val = np.ones(labels.shape)
    power0 = power0.swapaxes(0,1)
    power1 = power1.swapaxes(0,1)
    data = np.concatenate((subj,labels,channels,num_elecs,da_mean,da_max,
                           power0, power1, time_vec, x_val),axis=0)
    df = pd.DataFrame(data.T, columns=['subjects','labels','channels','num_elecs','da_mean',
                                       'da_max','pow0','pow1','time','counter'])
    print(freq, 'nb of sig elecs', df.shape[0])
    sig_rois = np.unique(labels)
    olf_rois = ['pPirT','OFC_olf','Amg']
    for roi in sig_rois:
        df_roi_f = df.loc[df['labels'] == roi]
        df_roi_f['pow0'] = df_roi_f['pow0'].astype(float)
        df_roi_f['pow1'] = df_roi_f['pow1'].astype(float)
        df_roi_f['sign'] = np.sign(df_roi_f['pow1'] - df_roi_f['pow0'])
        inc = (df_roi_f.loc[df_roi_f.sign == 1.0]).shape[0]
        dec = (df_roi_f.loc[df_roi_f.sign == -1.0]).shape[0]
        
        df_inc = df_roi_f.loc[df_roi_f.sign == 1.0].groupby(['subjects']).count()
        df_dec = df_roi_f.loc[df_roi_f.sign == -1.0].groupby(['subjects']).count()
        
        if (df_inc.shape[0] >= 3) or (df_inc.shape[0] >=2 and roi in olf_rois):
            if not exists(path_to_save.format(str(th),str(min_win),roi)):
                makedirs(path_to_save.format(str(th),str(min_win),roi))
            print(roi,'%s electrodes showed increased TPSim, while %s showed decrease out of %s elecs' 
                  % (inc,dec,inc+dec))
            df_plot = df_roi_f[['subjects','labels','channels','num_elecs','da_max',
                                'sign','time']].loc[df_roi_f.sign == 1.0]
            df_plot.to_csv(save_csv.format(str(th),str(min_win),roi,roi,'inc',str(th),str(min_win)))
            
        if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_rois):
            if not exists(path_to_save.format(str(th),str(min_win),roi)):
                makedirs(path_to_save.format(str(th),str(min_win),roi))
            print(roi,'%s electrodes showed increased TPSim, while %s showed decrease out of %s elecs' 
                  % (inc,dec,inc+dec))
            df_plot = df_roi_f[['subjects','labels','channels','num_elecs','da_max',
                                'sign','time']].loc[df_roi_f.sign == -1.0]
            df_plot.to_csv(save_csv.format(str(th),str(min_win),roi,roi,'dec',str(th),str(min_win)))
        

-> Olfacto loaded
gamma_high nb of sig elecs 35
IFG 7 electrodes showed increased TPSim, while 1 showed decrease out of 8 elecs
OFC_olf 1 electrodes showed increased TPSim, while 4 showed decrease out of 5 elecs


In [56]:
"""
Find permutation threshold when selecting only SPECIFIC ROIS
"""
from brainpipe.system import study
from os import listdir, makedirs
from os.path import isfile, join, exists
from brainpipe.statistics import *
import numpy as np
from itertools import product
import seaborn as sns
from collections import Counter

conds = ['low','high']
exp = 'Enc'
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/0_LDA_Pow_'+exp[0]+'_by_cond_1_2s/')
path_to_save = join(path_npz, 'masks_stat/')
f_form = '{}_sources_{}_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
f_save = '{}_mask_stat_{}_minwin{}_th{}_corr_{}_Olf.npy'
f_save = join(path_to_save,f_save)
###############################################################################
if not exists(path_to_save):
    makedirs(path_to_save)
###############################################################################  
subjects = ['CHAF','FERJ','LEFC','SEMC','PIRJ','VACJ'] #CHAF
freqs = ['0_theta', '1_alpha','2_beta','3_gamma']
th = 0.05
n_perms = 1000
min_win = 1.

#rois_to_keep = ['ACC','HC','IFG','Ins','MFG','OFC','PHG','SFG','pPirT']
rois_to_keep = ['HC','OFC','pPirT','Amg','Amg-pPirT']

# #Find threshold by subject (corrected for regions and freqs)
th_by_su = []
for su in sorted(subjects):
    th_su = []
    for freq in freqs:
        mat = np.load(f_form.format('All_subjects',freq))
        id_su = np.where(mat['su_codes']==su)
        id_lab = [i for i,x in enumerate(mat['s_labels'][id_su]) if x in rois_to_keep]
        s_perm = mat['s_perm'][id_su][id_lab].swapaxes(0,1)
        if s_perm.shape[1] > 0:
            th_perm = perm_pvalue2level(s_perm, p=th, maxst=True)[0]
        else:
            th_perm = 1.
        th_su.append(th_perm)
    max_su = max(th_su)
    th_by_su.append(max_su)
print('perm max size',th_by_su)

# Create masks for a specific perm threshold
for f,freq in enumerate(freqs):
    mat = np.load(f_form.format('All_subjects',freq))    
    mask_f, labels = np.array([]),[]
    for i,su in enumerate(sorted(subjects)):
        id_su = np.where(mat['su_codes']==su)
        if np.size(mat['s_da'][1]) > 1 and freq in ['0_theta', '1_alpha','2_beta','3_gamma']:
            s_da = mat['s_da'][id_su][:,10:30]
        elif np.size(mat['s_da'][1]) > 1 and freq in ['4_ripples']:
            s_da = mat['s_da'][id_su][:,7:]
        else:
            s_da = mat['s_da'][id_su][:,np.newaxis]
        s_labels = mat['s_labels'][id_su]
        id_lab = [i for i,x in enumerate(s_labels) if x in rois_to_keep]
        s_da = s_da[id_lab]
        s_labels = s_labels[id_lab]
        
        #select ROIS to compute
        n_elecs, npts = s_da.shape
        mask_su = []
        for elec in range(n_elecs):
            da_elec = s_da[elec]
            label = s_labels[elec]
            pvals = np.array([])
            for t in range(npts):
                if da_elec[t] > th_by_su[i]:
                    pval = 1
                    labels.append(label)
                if da_elec[t] <= th_by_su[i]:
                    pval = 0
                pvals = np.hstack((pvals,pval)) if np.size(pvals) else pval
            underp = np.where(pvals == 1)[0]
            pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
            signif = [True for k in pvsplit if len(k) >= min_win]
            if len(signif) >=1:
                remove = False
            else:
                remove = True
            mask_su.append(remove)
        mask_f = np.hstack((mask_f, mask_su)) if np.size(mask_f) else mask_su
    print(freq, len(mask_f),len(np.where(mask_f == False)[0]),Counter(labels))
    np.save(f_save.format('All_subjects',freq,min_win,th,'all'),mask_f)

-> Olfacto loaded
perm max size [1.0, 0.8, 0.7111904761904763, 0.8111111111111111, 0.7928571428571429, 0.8111111111111111]
0_theta 104 16 Counter({'OFC': 7, 'HC': 5, 'pPirT': 4})
1_alpha 104 17 Counter({'HC': 10, 'OFC': 4, 'pPirT': 3})
2_beta 104 18 Counter({'HC': 10, 'pPirT': 4, 'OFC': 4})
3_gamma 104 25 Counter({'HC': 10, 'OFC': 9, 'pPirT': 6})
